In [3]:
import pandas as pd

df = pd.read_csv('../data/sales.csv')
df.head(5)

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18


In [4]:
unique_store_ids = df['store_id'].nunique()
unique_cat_ids = df['cat_id'].nunique()

print(f"Number of unique store_id values: {unique_store_ids}")
print(f"Number of unique cat_id values: {unique_cat_ids}")

Number of unique store_id values: 10
Number of unique cat_id values: 3


In [7]:
# Verify the date range of the sales data
df['date'] = pd.to_datetime(df['date'])
date_range_start = df['date'].min()
date_range_end = df['date'].max()

print(f"Sales data starts from: {date_range_start}")
print(f"Sales data ends on: {date_range_end}")

Sales data starts from: 2011-01-29 00:00:00
Sales data ends on: 2016-05-22 00:00:00


In [8]:
# Check for duplicated dates within a store and category group
duplicated_dates = df[df.duplicated(subset=['date', 'store_id', 'cat_id'], keep=False)]

if not duplicated_dates.empty:
    print("Duplicated dates found within store and category groups:")
    print(duplicated_dates)
else:
    print("No duplicated dates found within store and category groups.")


No duplicated dates found within store and category groups.


In [9]:
# Check for missing dates in the sales data
all_dates = pd.date_range(start=date_range_start, end=date_range_end)
missing_dates = all_dates.difference(df['date'].unique())

if not missing_dates.empty:
    print("Missing dates in the sales data:")
    print(missing_dates)
else:
    print("No missing dates in the sales data.")


No missing dates in the sales data.


In [11]:
from scipy.signal import detrend

# Detrend the sales data
df['detrended_sales'] = detrend(df['sales'])

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['detrended_sales'].quantile(0.25)
Q3 = df['detrended_sales'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df[(df['detrended_sales'] < lower_bound) | (df['detrended_sales'] > upper_bound)]

if not outliers.empty:
    print("Outliers detected in the sales data:")
    print(outliers)
else:
    print("No outliers detected in the sales data.")


/Users/pedro/miniconda3/envs/live/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Outliers detected in the sales data:
            date store_id cat_id     sales  detrended_sales
1598  2015-06-15     TX_1  FOODS  10036.83      6405.333363
6251  2012-04-01     TX_2  FOODS   9969.98      6396.227490
6257  2012-04-07     TX_2  FOODS  10122.12      6548.441950
6286  2012-05-06     TX_2  FOODS  10007.18      6433.861843
6783  2013-09-15     TX_2  FOODS  10379.24      6812.089655
...          ...      ...    ...       ...              ...
54304 2016-04-09     WI_3  FOODS  10419.92      7442.509310
54305 2016-04-10     WI_3  FOODS   9259.20      6281.801720
54332 2016-05-07     WI_3  FOODS   9494.98      6517.916792
54339 2016-05-14     WI_3  FOODS  10608.35      7631.373663
54340 2016-05-15     WI_3  FOODS  11517.60      8540.636073

[1623 rows x 5 columns]


In [12]:
# Check for missing sales values in the sales data
missing_sales = df['sales'].isna()

if missing_sales.any():
    print("Missing sales values detected in the sales data:")
    print(df[missing_sales])
else:
    print("No missing sales values in the sales data.")


No missing sales values in the sales data.


In [13]:
# Set outliers to null value
df.loc[(df['detrended_sales'] < lower_bound) | (df['detrended_sales'] > upper_bound), 'sales'] = None

# Impute the missing sales using the linear interpolation method
df['sales'] = df['sales'].interpolate(method='linear')

# Verify if there are still any missing values after interpolation
if df['sales'].isna().any():
    print("There are still missing sales values after interpolation.")
else:
    print("All missing sales values have been imputed using linear interpolation.")


All missing sales values have been imputed using linear interpolation.


In [14]:
# Save the preprocessed data to a CSV file
df.to_csv('../data/sales_processed.csv', index=False)
